In [3]:

!pip install transformers datasets torch scikit-learn
!pip install --upgrade torch torchvision torchaudio

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: C:\Users\voitu\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


^C



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: C:\Users\voitu\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
ERROR: Exception:
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2544.0_x64__qbz5n2kfra8p0\Lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2544.0_x64__qbz5n2kfra8p0\Lib\site-packages\pip\_vendor\urllib3\response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
           ^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2544.0_x64__qbz5n2kfra8p0\Lib\site-packages\pip\_vendor\urllib3\response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
           ^^^^

Defaulting to user installation because normal site-packages is not writeable
  Using cached torch-2.6.0-cp312-cp312-win_amd64.whl.metadata (28 kB)
  Using cached torchvision-0.21.0-cp312-cp312-win_amd64.whl.metadata (6.3 kB)
  Using cached torchaudio-2.6.0-cp312-cp312-win_amd64.whl.metadata (6.7 kB)
   ---------------------------------------- 0.0/204.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/204.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/204.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/204.1 MB ? eta -:--:--
   ---------------------------------------- 0.3/204.1 MB ? eta -:--:--
   ---------------------------------------- 0.3/204.1 MB ? eta -:--:--
   ---------------------------------------- 0.3/204.1 MB ? eta -:--:--
   ---------------------------------------- 0.5/204.1 MB 453.5 kB/s eta 0:07:29
   ---------------------------------------- 0.5/204.1 MB 453.5 kB/s eta 0:07:29
   --------------------------------------

In [ ]:

# 📌 Importation des bibliothèques
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback, get_scheduler
from datasets import Dataset
# from google.colab import drive
import os

# 📌 Vérifier si un GPU est disponible
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Utilisation de : {device}")

# 📌 Charger les données depuis Hugging Face

splits = {'train': 'default/train/train.parquet', 'validation': 'default/validation/validation.parquet', 'test': 'default/test/test.parquet'}
df = pd.read_parquet("hf://datasets/TaumeV/flirty_or_not_IA_extended/" + splits["train"])

df_train = pd.read_parquet("hf://datasets/TaumeV/flirty_or_not_IA_extended/" + splits["train"])
df_val = pd.read_parquet("hf://datasets/TaumeV/flirty_or_not_IA_extended/" + splits["validation"])
df_train = df_train.drop("id", axis=1)
df_val = df_val.drop("id", axis=1)

# 📌 Vérification des données
print(f"Train shape: {df_train.shape}, Validation shape: {df_val.shape}")

# 📌 Charger le tokenizer BERT
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# 📌 Fonction de tokenization
def preprocess_function(examples):
    return tokenizer(examples["texts"], padding="max_length", truncation=True)

# 📌 Conversion en dataset Hugging Face
train_dataset = Dataset.from_pandas(df_train)
val_dataset = Dataset.from_pandas(df_val)

train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)

# 📌 Charger le modèle BERT avec Dropout amélioré
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=2,
    hidden_dropout_prob=0.3,  # Augmenté pour réduire l'overfitting
    attention_probs_dropout_prob=0.3
)
model.to(device)  # Envoyer sur GPU si disponible

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",  # Correction de la dépréciation de evaluation_strategy
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=50,  # Mettre à jour toutes les 50 étapes
    learning_rate=1e-5,  # Learning Rate réduit pour une meilleure convergence
    per_device_train_batch_size=32,  # Augmenté pour une meilleure stabilité
    per_device_eval_batch_size=32,
    num_train_epochs=5,  # Réduction des époques pour éviter l'overfitting
    weight_decay=0.01,
    report_to="none",
    load_best_model_at_end=True,  # ✅ Recharger le meilleur modèle à la fin
    metric_for_best_model="eval_loss"  # ✅ Définir la métrique d'évaluation pour EarlyStopping
)

# 📌 Création du Trainer avec Early Stopping
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]  # Stop après 2 époques sans amélioration
)

# 📌 Entraîner le modèle
trainer.train()

# 📌 Ajouter un Scheduler APRÈS l'entraînement
num_training_steps = len(train_dataset) * training_args.num_train_epochs
lr_scheduler = get_scheduler(
    name="linear", optimizer=trainer.optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

trainer.optimizer.param_groups[0]['lr_scheduler'] = lr_scheduler


# 📌 Obtenir le chemin du dossier Téléchargements de l'utilisateur
download_folder = os.path.join("C:/Users/Nom/", "Downloads", "bert_flirty_model_optimized2")

# 📌 Sauvegarde du modèle dans le dossier Téléchargements
model.save_pretrained(download_folder)
tokenizer.save_pretrained(download_folder)

print(f"Modèle sauvegardé dans : {download_folder}")



# 📌 Sauvegarde du modèle dans Google Drive (optionnel)
# drive.mount('/content/drive')

# model.save_pretrained("/content/drive/MyDrive/bert_flirty_model_optimized2")
# tokenizer.save_pretrained("/content/drive/MyDrive/bert_flirty_model_optimized2")

# print("Modèle optimisé sauvegardé avec succès dans Google Drive !")

KeyboardInterrupt: 

In [ ]:
from datasets import load_metric

# 📌 Charger la métrique d'accuracy
metric = load_metric("accuracy")

# 📌 Fonction de calcul des métriques
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)  # Convertir logits en classes
    return metric.compute(predictions=predictions.numpy(), references=labels)

# 📌 Mettre à jour le Trainer avec la métrique
trainer.compute_metrics = compute_metrics

# 📌 Évaluer le modèle sur l'ensemble de validation
results = trainer.evaluate()

# 📌 Afficher l'accuracy
print(f"Accuracy sur l'ensemble de validation: {results['eval_accuracy']:.4f}")